
- RowNumber
- CustomerId
- Surname
- CreditScore
- Geography
- Gender
- Age
- Tenure
- Balance
- NumOfProductsHow many accounts, bank account affiliated products the person has
- HasCrCardDoes the customer have a credit card through the bank?
- IsActiveMemberSubjective, but for the concept
- EstimatedSalary

In [1]:
!nvidia-smi

Sun Jan 12 17:53:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# Loading neccessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns

In [0]:
# Loading data from the drive folder

bank_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project/NN/bank.csv')

In [5]:
# Checking the shape of the data
bank_data.shape

(10000, 14)

In [59]:
# checking some records

bank_data.head(8)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1


In [60]:
# checking information
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [61]:
bank_data.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [62]:
# Checking the target column
bank_data['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [0]:
#RowNumber, CustomerId, Surname currently not relevant for the prediction
# Dropping the same.
bank_df = bank_data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=False)

In [64]:
bank_df.isna().sum()

# no missing value observed

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [65]:
bank_df.iloc[:, :-1]

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [0]:
# target 
y = bank_df.iloc[:, -1]

# Features
X = bank_df.iloc[:, :-1]

In [86]:
X.shape

(10000, 10)

In [87]:
y.shape

(10000,)

In [88]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [0]:
# converting Geography, Gender to categorical columns 
X['Geography'] = X['Geography'].astype('category')
X['Gender'] = X['Gender'].astype('category')

In [90]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null category
Gender             10000 non-null category
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
dtypes: category(2), float64(2), int64(6)
memory usage: 644.9 KB


In [9]:
X['Gender'].dtypes

CategoricalDtype(categories=['Female', 'Male'], ordered=False)

In [0]:
# X = pd.get_dummies(data=X, drop_first=True)

In [93]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [94]:
X.shape

(10000, 10)

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [0]:
ct = ColumnTransformer(
    [("cat", OneHotEncoder(drop='first'), [1, 2]),      
    ("norm1", StandardScaler(), [0,3,4,5,6,7,8,9])])

In [97]:
X_encoded.shape

(10000, 11)

In [0]:
# from sklearn.preprocessing import OneHotEncoder
# one_hot_Encoder = OneHotEncoder( categories=[1, 2],drop='first', sparse=False)


In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Flatten, Dense, Dropout, Reshape, MaxPool2D

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=3)

In [0]:
X_train_encoded = ct.fit_transform(X_train)

In [0]:

X_test_encoded = ct.transform(X_test)

In [0]:
X_train = tf.cast(X_train_encoded, tf.float32)
X_test = tf.cast(X_test_encoded, tf.float32)
y_train = tf.cast(y_train.values, tf.float32)
y_test = tf.cast(y_test.values, tf.float32)

In [20]:
X_train

<tf.Tensor: shape=(8000, 11), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ..., -1.5394285 ,
         0.97238123,  0.9428422 ],
       [ 1.        ,  0.        ,  1.        , ..., -1.5394285 ,
        -1.0284032 , -0.8892186 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.64959174,
        -1.0284032 , -1.243755  ],
       ...,
       [ 1.        ,  0.        ,  1.        , ...,  0.64959174,
         0.97238123,  1.4721545 ],
       [ 0.        ,  0.        ,  0.        , ..., -1.5394285 ,
         0.97238123,  1.0635045 ],
       [ 1.        ,  0.        ,  1.        , ...,  0.64959174,
         0.97238123, -1.2962918 ]], dtype=float32)>

In [0]:
# X_train = tf.data.Dataset.from_tensor_slices(X_train)
# X_test  = tf.data.Dataset.from_tensor_slices(X_test)
# y_train = tf.data.Dataset.from_tensor_slices(y_train)
# y_test  = tf.data.Dataset.from_tensor_slices(y_test)

In [0]:
# for feat in X_train.take(5):
#   print ('Features: {}'.format(feat))

In [0]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [21]:
X_train.shape

TensorShape([8000, 11])

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

In [63]:
# Initialize Sequential model
model_with_3_layers = Sequential()    

#Add 1st hidden layer
model_with_3_layers.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
model_with_3_layers.add(Dropout(rate=0.1)) 

#Add 2nd hidden layer
model_with_3_layers.add(Dense(6, activation = 'relu'))
model_with_3_layers.add(Dropout(rate=0.1)) 

#Add OUTPUT layer
model_with_3_layers.add(Dense(1, activation = 'sigmoid'))

# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile the model
model_with_3_layers.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

#summary
model_with_3_layers.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_1 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [64]:
# Initialize Sequential model
model_with_3_layers_2 = Sequential()    

#Add 1st hidden layer
model_with_3_layers_2.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
model_with_3_layers_2.add(Dropout(rate=0.1)) 

#Add 2nd hidden layer
model_with_3_layers_2.add(Dense(6, activation = 'relu'))
model_with_3_layers_2.add(Dropout(rate=0.1)) 

#Add OUTPUT layer
model_with_3_layers_2.add(Dense(1, activation = 'sigmoid'))

# Compile the model
model_with_3_layers_2.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

#summary
model_with_3_layers_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_3 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Initialize Sequential model
model_with_4_layers = Sequential()    

#Add 1st hidden layer
model_with_4_layers.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
model_with_4_layers.add(Dropout(rate=0.1)) 

#Add 2nd hidden layer
model_with_4_layers.add(Dense(6, activation = 'relu'))
model_with_4_layers.add(Dropout(rate=0.1)) 

#Add 3rd hidden layer
model_with_4_layers.add(Dense(6, activation = 'relu'))
model_with_4_layers.add(Dropout(rate=0.1)) 

#Add OUTPUT layer
model_with_4_layers.add(Dense(1, activation = 'sigmoid'))

# Define the optimizer
optimizer_sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compile the model
model_with_4_layers.compile(optimizer=optimizer_sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

#summary
model_with_4_layers.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 6)                 72        
_________________________________________________________________
dropout_4 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_5 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 42        
_________________________________________________________________
dropout_6 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [66]:
# Initialize Sequential model
model_with_4_layers_2 = Sequential()    

#Add 1st hidden layer
model_with_4_layers_2.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
model_with_4_layers_2.add(Dropout(rate=0.1)) 

#Add 2nd hidden layer
model_with_4_layers_2.add(Dense(6, activation = 'relu'))
model_with_4_layers_2.add(Dropout(rate=0.1)) 

#Add 3rd hidden layer
model_with_4_layers_2.add(Dense(6, activation = 'relu'))
model_with_4_layers_2.add(Dropout(rate=0.1)) 

#Add OUTPUT layer
model_with_4_layers_2.add(Dense(1, activation = 'sigmoid'))

# Compile the model
model_with_4_layers_2.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

#summary
model_with_4_layers_2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 6)                 72        
_________________________________________________________________
dropout_7 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 42        
_________________________________________________________________
dropout_8 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 42        
_________________________________________________________________
dropout_9 (Dropout)          (None, 6)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

In [67]:
model_with_3_layers_history = model_with_3_layers.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,  
                    batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 164us/sample - loss: 0.6145 - accuracy: 0.7053 - val_loss: 0.5173 - val_accuracy: 0.8025
Epoch 2/10
8000/8000 [==============================] - 1s 97us/sample - loss: 0.4990 - accuracy: 0.7947 - val_loss: 0.4633 - val_accuracy: 0.8045
Epoch 3/10
8000/8000 [==============================] - 1s 93us/sample - loss: 0.4724 - accuracy: 0.7989 - val_loss: 0.4440 - val_accuracy: 0.8110
Epoch 4/10
8000/8000 [==============================] - 1s 91us/sample - loss: 0.4590 - accuracy: 0.8051 - val_loss: 0.4317 - val_accuracy: 0.8175
Epoch 5/10
8000/8000 [==============================] - 1s 92us/sample - loss: 0.4436 - accuracy: 0.8116 - val_loss: 0.4195 - val_accuracy: 0.8205
Epoch 6/10
8000/8000 [==============================] - 1s 100us/sample - loss: 0.4332 - accuracy: 0.8120 - val_loss: 0.4081 - val_accuracy: 0.8270
Epoch 7/10
8000/8000 [==============================] - 1s 99us/samp

In [68]:
model_with_3_layers_2_history = model_with_3_layers_2.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,  
                    batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 136us/sample - loss: 0.5295 - accuracy: 0.7676 - val_loss: 0.4559 - val_accuracy: 0.8045
Epoch 2/10
8000/8000 [==============================] - 1s 97us/sample - loss: 0.4727 - accuracy: 0.7966 - val_loss: 0.4323 - val_accuracy: 0.8110
Epoch 3/10
8000/8000 [==============================] - 1s 100us/sample - loss: 0.4530 - accuracy: 0.8001 - val_loss: 0.4197 - val_accuracy: 0.8115
Epoch 4/10
8000/8000 [==============================] - 1s 118us/sample - loss: 0.4392 - accuracy: 0.8043 - val_loss: 0.4098 - val_accuracy: 0.8155
Epoch 5/10
8000/8000 [==============================] - 1s 98us/sample - loss: 0.4299 - accuracy: 0.8076 - val_loss: 0.4010 - val_accuracy: 0.8205
Epoch 6/10
8000/8000 [==============================] - 1s 94us/sample - loss: 0.4201 - accuracy: 0.8152 - val_loss: 0.3930 - val_accuracy: 0.8275
Epoch 7/10
8000/8000 [==============================] - 1s 92us/sam

In [51]:
model_with_4_layers_history = model_with_4_layers.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,  
                    batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 125us/sample - loss: 0.5752 - accuracy: 0.7945 - val_loss: 0.4431 - val_accuracy: 0.8025
Epoch 2/10
8000/8000 [==============================] - 1s 83us/sample - loss: 0.4422 - accuracy: 0.7947 - val_loss: 0.4337 - val_accuracy: 0.8025
Epoch 3/10
8000/8000 [==============================] - 1s 85us/sample - loss: 0.4346 - accuracy: 0.7947 - val_loss: 0.4319 - val_accuracy: 0.8025
Epoch 4/10
8000/8000 [==============================] - 1s 84us/sample - loss: 0.4320 - accuracy: 0.7947 - val_loss: 0.4303 - val_accuracy: 0.8025
Epoch 5/10
8000/8000 [==============================] - 1s 86us/sample - loss: 0.4303 - accuracy: 0.7947 - val_loss: 0.4290 - val_accuracy: 0.8025
Epoch 6/10
8000/8000 [==============================] - 1s 86us/sample - loss: 0.4288 - accuracy: 0.7947 - val_loss: 0.4276 - val_accuracy: 0.8025
Epoch 7/10
8000/8000 [==============================] - 1s 85us/sampl

In [69]:
model_with_4_layers_2_history = model_with_4_layers_2.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,  
                    batch_size=32) 

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 144us/sample - loss: 0.5461 - accuracy: 0.7860 - val_loss: 0.4672 - val_accuracy: 0.8025
Epoch 2/10
8000/8000 [==============================] - 1s 90us/sample - loss: 0.4805 - accuracy: 0.7956 - val_loss: 0.4444 - val_accuracy: 0.8025
Epoch 3/10
8000/8000 [==============================] - 1s 92us/sample - loss: 0.4636 - accuracy: 0.7976 - val_loss: 0.4299 - val_accuracy: 0.8105
Epoch 4/10
8000/8000 [==============================] - 1s 92us/sample - loss: 0.4506 - accuracy: 0.8064 - val_loss: 0.4158 - val_accuracy: 0.8220
Epoch 5/10
8000/8000 [==============================] - 1s 90us/sample - loss: 0.4308 - accuracy: 0.8086 - val_loss: 0.4044 - val_accuracy: 0.8250
Epoch 6/10
8000/8000 [==============================] - 1s 96us/sample - loss: 0.4255 - accuracy: 0.8125 - val_loss: 0.3981 - val_accuracy: 0.8260
Epoch 7/10
8000/8000 [==============================] - 1s 99us/sampl

In [0]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [0]:
# Compile the model
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,  
                    batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 157us/sample - loss: 0.2486 - accuracy: 0.8434 - val_loss: 2.6893 - val_accuracy: 0.8025
Epoch 2/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.2478 - accuracy: 0.8441 - val_loss: 2.8602 - val_accuracy: 0.8025
Epoch 3/10
8000/8000 [==============================] - 1s 159us/sample - loss: 0.2474 - accuracy: 0.8443 - val_loss: 3.0218 - val_accuracy: 0.8025
Epoch 4/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.2465 - accuracy: 0.8451 - val_loss: 3.1699 - val_accuracy: 0.8025
Epoch 5/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.2462 - accuracy: 0.8454 - val_loss: 3.3408 - val_accuracy: 0.8025
Epoch 6/10
8000/8000 [==============================] - 1s 158us/sample - loss: 0.2458 - accuracy: 0.8456 - val_loss: 3.4415 - val_accuracy: 0.8025
Epoch 7/10
8000/8000 [==============================] - 1s 163us

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

In [0]:
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate=0.1)) 